In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 10:11:44 2022

@author: roy.369
"""


import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit
from matplotlib.ticker import ScalarFormatter

In [2]:
def arr_loc(val,X):
   ind = 0
   for i in range(len(X)):
      if(round(X[i],2) == round(val,2)):
         ind = i
         break
   return ind


def power_fit(x,a1,m):
    return a1*x**m
  
def linear_fit(x,a,b):
    return a+b*x
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [3]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1

In [4]:
def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [5]:
def k_point_grid_upper_half_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              K_lab.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


def k_point_grid_full_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_lab.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


In [6]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_upper_half_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [12]:
def plot_func_multiple_beta_hf(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(["red","orange","goldenrod","blue","purple"]) #(cm.gnuplot(np.linspace(0, 1, len(trot)+1)))
   plt.figure(figsize = (25,20))
   plt.xticks([2,4,6,8,10],fontsize = 120)
   plt.yticks([0,-2,-4,-6],fontsize = 120)
   plt.xlim(0,12)
   #plt.ylim(-7,1) 
   plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
   plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
   #Hd_fit = np.zeros(len(U_val))
   #plt.tick_params(axis='both', which='both', direction='in', pad=20)
   for jj in range(len(trot)):
       c_1 = next(color_1)
       beta = (float(dtau))*(float(trot[jj]))
       T = round(1/beta,2)
       plt.errorbar(x_val,y_val[:,jj]*10**3,yerr = y_val_err[:,jj]*10**3,c=c_1,marker = 'o',markersize = 25, elinewidth = 3, capsize = 5,label = r"T = %s"%str(T))
   plt.grid(True,which='both')

   plt.axvspan(3.1, 4.3, facecolor='r', alpha=0.2)
   #plt.legend(loc = 'best',fontsize = 80)
   plt.tight_layout()
   plt.savefig(plot_name)
   plt.close()

In [13]:
def equal_time_density_density_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm,Graph_dir_hf,N,u,mu,dtau,L):

   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   Den_Den_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Den_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Den_Den_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Den_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))

    
   Nden = np.zeros((len(u),len(L)))
   Nden_std = np.zeros((len(u),len(L)))
    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       
       
   for i in range(len(u)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,mu,dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[i][k] = sys_measure_avg['Density averaged']
           Nden_std[i][k] = sys_measure_std['Density standard deviation']
           
           Text_dir_den_den_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])
           Text_dir_den_den_conn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_connected_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])

           filename_den_den_corr = '%s/Equal_time_Density_Density_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Den_corr_hf[i,k,:],Den_Den_corr_hf_std[i,k,:] = np.loadtxt(filename_den_den_corr,unpack = 'True', usecols = [1,2])

           filename_den_den_conn_corr = '%s/Equal_time_Density_Density_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_conn_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Den_conn_corr_hf[i,k,:],Den_Den_conn_corr_hf_std[i,k,:] = np.loadtxt(filename_den_den_conn_corr,unpack = 'True', usecols = [1,2])


 
   Den_Den_corr_hf_n0 = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n1 = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n2 = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n3 = np.zeros((len(u),len(L)))

   Den_Den_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n3_std = np.zeros((len(u),len(L)))

   Den_Den_conn_corr_hf_n0 = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n1 = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n2 = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n3 = np.zeros((len(u),len(L)))
    
   Den_Den_conn_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n3_std = np.zeros((len(u),len(L)))


   for i in range(len(u)):
       for k in range(len(L)):

           Den_Den_corr_hf_n0[i][k] = Den_Den_corr_hf[i][k][0]
           Den_Den_corr_hf_n1[i][k] = Den_Den_corr_hf[i][k][1]
           Den_Den_corr_hf_n2[i][k] = Den_Den_corr_hf[i][k][2]
           Den_Den_corr_hf_n3[i][k] = Den_Den_corr_hf[i][k][3]

           Den_Den_corr_hf_n0_std[i][k] = Den_Den_corr_hf_std[i][k][0]
           Den_Den_corr_hf_n1_std[i][k] = Den_Den_corr_hf_std[i][k][1]
           Den_Den_corr_hf_n2_std[i][k] = Den_Den_corr_hf_std[i][k][2]
           Den_Den_corr_hf_n3_std[i][k] = Den_Den_corr_hf_std[i][k][3]

           Den_Den_conn_corr_hf_n0[i][k] = Den_Den_conn_corr_hf[i][k][0]
           Den_Den_conn_corr_hf_n1[i][k] = Den_Den_conn_corr_hf[i][k][1]
           Den_Den_conn_corr_hf_n2[i][k] = Den_Den_conn_corr_hf[i][k][2]
           Den_Den_conn_corr_hf_n3[i][k] = Den_Den_conn_corr_hf[i][k][3]
           
           Den_Den_conn_corr_hf_n0_std[i][k] = Den_Den_conn_corr_hf_std[i][k][0]
           Den_Den_conn_corr_hf_n1_std[i][k] = Den_Den_conn_corr_hf_std[i][k][1]
           Den_Den_conn_corr_hf_n2_std[i][k] = Den_Den_conn_corr_hf_std[i][k][2]
           Den_Den_conn_corr_hf_n3_std[i][k] = Den_Den_conn_corr_hf_std[i][k][3]
           
           
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"

    
   Pl_name_n1 = "%s/Density_density_correlation_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n1 = "%s/Density_density_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n2 = "%s/Density_density_correlation_connected_2nd_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)

  
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,U_val,Den_Den_corr_hf_n1,Den_Den_corr_hf_n1_std,Pl_name_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Den_Den_conn_corr_hf_n1,Den_Den_conn_corr_hf_n1_std,Pl_name_conn_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Den_Den_conn_corr_hf_n2,Den_Den_conn_corr_hf_n2_std,Pl_name_conn_n2)
        



In [14]:
def density_susceptibility_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm,Graph_dir_hf,N,u,mu,dtau,L):

   Adj_list =adj_mat_calc(N)
    
   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))
   
   Den_Den_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Den_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Den_Den_conn_corr_nl_hf = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_nl_hf_std = np.zeros((len(u),len(L)))
    
   Nden = np.zeros((len(u),len(L)))
   Nden_std = np.zeros((len(u),len(L)))

   Kappa_hf = np.zeros((len(u),len(L)))
   Kappa_hf_std = np.zeros((len(u),len(L))) 
    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       
   T_val = np.zeros(len(L))
   for k in range(len(L)):
       T_val[k] = 1/(float(dtau)*float(L[k]))
       
   for i in range(len(u)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,mu,dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[i][k] = sys_measure_avg['Density averaged']
           Nden_std[i][k] = sys_measure_std['Density standard deviation']

           Text_dir_den_den_conn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_connected_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])

           filename_den_den_conn_corr = '%s/Equal_time_Density_Density_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_conn_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Den_conn_corr_hf[i,k,:],Den_Den_conn_corr_hf_std[i,k,:] = np.loadtxt(filename_den_den_conn_corr,unpack = 'True', usecols = [1,2])

           #Den_Den_conn_corr_nl_hf[i,k] = 


           
           Den_Den_conn_corr_nl_hf_std[i,k] = np.sqrt(np.dot(np.power(Den_Den_conn_corr_hf_std[i,k,1:],2),Adj_list[1:,1]))

           Kappa_hf[i][k] = (1/T_val[k])*np.dot(Den_Den_conn_corr_hf[i,k,:],Adj_list[:,1])
           Kappa_hf_std[i,k] = (1/T_val[k])*np.sqrt(np.dot(np.power(Den_Den_conn_corr_hf_std[i,k,:],2),Adj_list[:,1]))
            
           
           
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"

    
   Pl_name_nl = "%s/Density_density_correlation_non_local_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_tot = "%s/Density_density_correlation_connected_fluctuations_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)

  
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,U_val,Den_Den_conn_corr_nl_hf,Den_Den_conn_corr_nl_hf_std,Pl_name_nl)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Kappa_hf,Kappa_hf_std,Pl_name_tot)
    



In [15]:
def main():

    N = "10"
    Dtau = "0.05"

    U = ["1.00","1.50","2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00","7.50","8.00","8.50","9.00","9.50","10.00","10.50","11.00","11.50","12.00"]
    Mu = "0.00"
    Trot = ["20","30","40","50","60"] 
    
    Text_dir_main_corr = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations_half_filling'%N
    Text_dir_main_eqm = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_half_filling'%N
    Graph_dir_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_half_filling/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged/Density_density_correlations'%(N,N,Dtau)
    if not os.path.exists(Graph_dir_hf):
        os.makedirs(Graph_dir_hf)

    Graph_dir = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged/Density_density_correlations'%(N,N,Dtau)
    if not os.path.exists(Graph_dir):
        os.makedirs(Graph_dir)
        
    equal_time_density_density_correlation_function_half_filling(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir_hf,N,U,Mu,Dtau,Trot)
    density_susceptibility_half_filling(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir_hf,N,U,Mu,Dtau,Trot)





In [16]:
main()